Author Joshua, Sudeep <br >
Summary <br> smh dude. This code generates the distance matrix? by loading texas data and open routing service api. In the end we output "gridinfosmaller.json
1. load texas data (census_tracts_2018_fips / census_tracts_2018)
2. texas data -> latitude longitude
3. latitude longitude -> distance matrix api calls


In [1]:
# unzip input file first
import xml.etree.ElementTree as ET
import requests
import math
import pprint
import time
import numpy as np
import pandas as pd
import os
import collections

### automate how big the grid is

In [2]:
# Assuming we want to make 400 squares, how should we partition the vertically/horizontally
NUM_REGIONS = 3200


In [3]:
# Read all census tracts and put points in dict
# CENSUS_TRACTS_DIR = "census_tracts"
CENSUS_TRACTS_DIR = "census_tracts_2018"
census_tracts_points = {} #multiple points?
census_tracts_bounding = {} #rectangle bounds?
for fname in os.listdir(CENSUS_TRACTS_DIR):
    latitudes = []
    longitudes = []
    try:
        # Parse the XML file
        root = ET.parse("{}/{}".format(CENSUS_TRACTS_DIR, fname)).getroot()
        for type_tag in root.findall("node"):
            d = type_tag.attrib
            latitudes.append(float(d["lat"]))
            longitudes.append(float(d["lon"]))
        census_tracts_points[fname.strip(".txt")] = list(zip(latitudes, longitudes))
        census_tracts_bounding[fname.strip(".txt")] = [(min(latitudes), min(longitudes)), (max(latitudes), max(longitudes))]
    except Exception as e:
        print(e)
        print(fname)
        print(list(zip(latitudes, longitudes)))

min() arg is an empty sequence
14_03.txt
[]
min() arg is an empty sequence
15_01.txt
[]
min() arg is an empty sequence
15_03.txt
[]
min() arg is an empty sequence
16_05.txt
[]
min() arg is an empty sequence
17_07.txt
[]
min() arg is an empty sequence
17_12.txt
[]
min() arg is an empty sequence
17_22.txt
[]
min() arg is an empty sequence
17_29.txt
[]
min() arg is an empty sequence
17_49.txt
[]
min() arg is an empty sequence
17_53.txt
[]
min() arg is an empty sequence
17_55.txt
[]
min() arg is an empty sequence
17_56.txt
[]
min() arg is an empty sequence
17_61.txt
[]
min() arg is an empty sequence
17_76.txt
[]
min() arg is an empty sequence
17_77.txt
[]
min() arg is an empty sequence
17_81.txt
[]
min() arg is an empty sequence
17_86.txt
[]
min() arg is an empty sequence
18_05.txt
[]
min() arg is an empty sequence
18_06.txt
[]
min() arg is an empty sequence
18_19.txt
[]
min() arg is an empty sequence
18_32.txt
[]
min() arg is an empty sequence
18_41.txt
[]
min() arg is an empty sequence
1

In [4]:
census_tracts_points;

## Travis County Boundary API

In [7]:
# root = ET.parse('input-files/RoughlyTravisCounty.osm').getroot()

In [8]:
latitudes = []
longitudes = []
# for type_tag in root.findall("node"):
#     d = type_tag.attrib
#     latitudes.append(float(d["lat"]))
#     longitudes.append(float(d["lon"]))
# min_lat_travis, max_lat_travis, min_lon_travis, max_lon_travis = float("inf"), float("-inf"), float("inf"), float("-inf")
for tract_name in census_tracts_points:
    for lat, lon in census_tracts_points[tract_name]:
#         if lat < min_lat_travis:
#             min_lat_travis = lat
#         if lon < min_lon_travis:
#             min_lon_travis = lon
#         if lat > max_lat_travis:
#             max_lat_travis = lat
#         if lon > max_lon_travis:
#             max_lon_travis = lon
        latitudes.append(lat)
        longitudes.append(lon)

## Region Mapping

In [9]:
LATITUDE_MILES = 69
LONGITUDE_MILES = 54.6

vertical_distance = (max(latitudes) - min(latitudes))*LATITUDE_MILES
horizontal_distance = (max(longitudes) - min(longitudes))*LONGITUDE_MILES

In [10]:
# Assuming we want to make 400 squares, how should we partition the vertically/horizontally
# NUM_REGIONS = 200
# square_len = (vertical_distance*horizontal_distance/NUM_REGIONS)**0.5

In [11]:
square_len = (vertical_distance*horizontal_distance/NUM_REGIONS)**0.5
square_len

2.1323175834794137

In [12]:
latitude_delta = square_len/LATITUDE_MILES
LATITUDE_DELTA = square_len/LATITUDE_MILES
longitude_delta = square_len/LONGITUDE_MILES
LONGITUDE_DELTA = square_len/LONGITUDE_MILES
longitude_delta

0.039053435594861054

In [104]:
lat_lon = zip(latitudes, longitudes) #joins lists

In [105]:
min_lat = min(latitudes)
min_lon = min(longitudes)
max_lat = max(latitudes)
max_lon = max(longitudes)
nrows = math.ceil((max_lat - min_lat) / latitude_delta)
ncols = math.ceil((max_lon - min_lon) / longitude_delta)
def map_point_to_region(latitude, longitude):
    return math.floor((latitude-min_lat)/latitude_delta) * ncols  + math.floor((longitude-min_lon)/longitude_delta)

In [106]:
def get_representative(region_num): # 8 cols per row, # R10 --> row_num = 1, col_num = 2
    row_num = region_num//ncols # Correct
    col_num = region_num - row_num*ncols
    lat = min_lat + row_num * latitude_delta + 0.5*latitude_delta
    lon = min_lon + col_num * longitude_delta + 0.5*longitude_delta
    return [lon, lat]

In [107]:
map_point_to_region(30.084446, -97.702885)

32

In [108]:
get_representative(13)

[-97.64344371946936, 30.04071777669188]

In [16]:
valid_regions = set()
for lat, lon in zip(latitudes, longitudes):
    valid_regions.add(map_point_to_region(lat, lon))

In [17]:
valid_regions = list(valid_regions)
valid_regions.sort()

In [18]:
region_coords = [get_representative(i) for i in range(NUM_REGIONS)]

## Batch calls of distance matrix

In [109]:
# Prepare data for api call
locations = region_coords
# locations = region_coords + ems_coords + hospital_coords
durations = [[0 for i in range(200)] for j in range(NUM_REGIONS)]

In [110]:
batch_size = 1
num_batches = math.ceil(len(valid_regions) / batch_size)
num_batches_per_round = math.ceil(num_batches/4)
current_branch = 3
remainder = len(valid_regions) % batch_size
print(num_batches)

1597


In [115]:
print(len(durations))
print(len(durations[0]))

400
200


###  (API Key needed (https://openrouteservice.org/))

In [111]:
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf6248f4d896877baa45689a97ee0efab1d43b',
    'Content-Type': 'application/json; charset=utf-8'
}

In [22]:
# for i in range(num_batches_per_round*current_branch, min(num_batches_per_round*current_branch+num_batches_per_round, len(valid_regions))):
#     print("batch ", i)
#     start = i * batch_size
#     sources = list(range(start, start + batch_size))

#     # handle last partial batch
#     if(sources[batch_size - 1] >= len(locations)):
#         sources = sources[:remainder]

#     locations = [get_representative(valid_regions[j]) for j in range(len(valid_regions))]

#     body = {"locations": locations, "sources": sources}
#     call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

#     duration_batch = call.json()['durations']

#     for j in range(len(duration_batch)):
#         for k in range(len(duration_batch[0])):
#             durations[valid_regions[i*batch_size + j]][valid_regions[k]] = duration_batch[j][k]
#     time.sleep(0.1)


In [23]:
#ONE API CALL FOR DEBUGGING
# locations = [get_representative(i) for i in range(NUM_REGIONS)]

# durations = []
# start = 1 * 8
# sources = list(range(start, start + 8))
# body = {"locations": locations, "sources": sources}
# call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)
# call.json()
# duration_batch = call.json()['durations']
# for j in range(len(duration_batch)):
#     durations.append(duration_batch[j])
# durations

400

In [455]:
""" new code """
"""doing 1x1 queries is 400x400 calls"""
"""doing 20x20 queries is 400 calls"""
#you have to index code slowly or you call make too many api calls a minute. Only 40 per minute
durations = np.zeros([NUM_REGIONS,NUM_REGIONS])
locations = [get_representative(i) for i in range(NUM_REGIONS)]

len(locations)
batch_size = 20
for i in range(0,int(NUM_REGIONS / batch_size)):
    for j in range(int(NUM_REGIONS / batch_size)):
        time.sleep(2)
        #if(i == j):
        x_start = i*batch_size
        y_start = j*batch_size
        print(str(x_start) + " " + str(y_start))
        location_batch = locations[x_start:x_start+batch_size] + locations[y_start:y_start+batch_size]
        sources = list(range(0, batch_size))
        body = {"locations": location_batch, "sources": sources}
        #print(len(body["locations"]))
        call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)
        #print(call.json())
        duration_batch = call.json()['durations']
        for j in range(len(duration_batch)):
            durations[y_start:y_start+batch_size, x_start+j] = duration_batch[j][batch_size:]
            #print(duration_batch[j])
            #print("newline")
        

0 0


KeyError: 'durations'

In [465]:
i = 0
j = 0
batch_size = 25
x_start = i*batch_size
y_start = j*batch_size
print(str(x_start) + " " + str(y_start))
location_batch = locations[x_start:x_start+batch_size] + locations[y_start:y_start+batch_size]
sources = list(range(0, batch_size))
body = {"locations": location_batch, "sources": sources}
#print(len(body["locations"]))
call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)
duration_batch = call.json()['durations']
for j in range(len(duration_batch)):
    durations[y_start:y_start+batch_size, x_start+j] = duration_batch[j][batch_size:]
    print(duration_batch[j])
    #print("newline")

0 0
[0.0, 961.57, 2006.0, 1703.25, 1966.77, 2196.37, 3745.08, 2974.18, 3182.53, 3505.84, 3856.11, 4228.19, 4218.59, 4478.41, 4731.4, 5013.93, 4969.52, 5837.53, 5408.04, 5788.88, 6262.38, 2380.21, 1870.67, 1577.84, 2831.08, 0.0, 961.57, 2006.0, 1703.25, 1966.77, 2196.37, 3745.08, 2974.18, 3182.53, 3505.84, 3856.11, 4228.19, 4218.59, 4478.41, 4731.4, 5013.93, 4969.52, 5837.53, 5408.04, 5788.88, 6262.38, 2380.21, 1870.67, 1577.84, 2831.08]
[961.57, 0.0, 1206.35, 903.6, 1167.13, 1396.72, 2783.55, 2174.53, 2382.89, 2706.19, 3056.47, 3428.54, 3418.94, 3678.76, 3931.76, 4214.28, 4169.87, 5037.88, 4608.39, 4989.23, 5384.97, 1690.63, 909.14, 616.31, 2031.43, 961.57, 0.0, 1206.35, 903.6, 1167.13, 1396.72, 2783.55, 2174.53, 2382.89, 2706.19, 3056.47, 3428.54, 3418.94, 3678.76, 3931.76, 4214.28, 4169.87, 5037.88, 4608.39, 4989.23, 5384.97, 1690.63, 909.14, 616.31, 2031.43]
[2006.0, 1206.35, 0.0, 456.56, 720.08, 949.68, 2718.52, 1727.49, 1935.84, 2259.15, 2609.42, 2981.5, 2971.89, 3231.72, 3484.71,

In [463]:
len(location_batch)

100

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

uniform_data = np.random.rand(400, 400)
#ax = sns.heatmap(durations, linewidth=0.00001)
ax = sns.heatmap(uniform_data, linewidth=0.00001)
plt.show()

In [119]:
locations = [get_representative(i) for i in range(NUM_REGIONS)]
len(locations)

400

In [2]:
locations = [get_representative(i) for i in range(NUM_REGIONS)]

durations = []

for i in range(25):
    start = i * 8
    sources = list(range(start, start + 8))
    body = {"locations": locations, "sources": sources}
    
    call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)
    
    duration_batch = call.json()['durations']
    for j in range(len(duration_batch)):
        durations.append(duration_batch[j])
        
    time.sleep(0.1)
    
pprint.pprint(durations)


NameError: name 'NUM_REGIONS' is not defined

In [468]:
len(durations)

200

In [127]:
#This is setting the time from region i to i. By default we set it to 5 minutes.
for i in range(len(durations)):
    for j in range(len(durations[0])):
        if i != j:
            continue
        durations[i][j] = 300


In [128]:
for i in range(len(durations)):
    for j in range(len(durations[0])):
        if durations[i][j] is None:
            print(i,j)
            durations[i][j] = 300

In [129]:
len(durations)

200

In [ ]:
#never used?
#def in_grid(grid_lat, grid_lon, lat, lon):
#    return (grid_lat <= lat <= grid_lat + LATITUDE_DELTA and grid_lon <= lon <= grid_lon + LONGITUDE_DELTA)

In [130]:
census_tract_grids = collections.defaultdict(set)
visited = set()
for tract_name in census_tracts_points:
    for lat, lon in census_tracts_points[tract_name]:
        region = map_point_to_region(lat, lon)
#         if region not in visited:
        census_tract_grids[tract_name].add(region)
#         visited.add(region)
    census_tract_grids[tract_name] = list(census_tract_grids[tract_name])

### This is where we export the json

In [131]:
import json

d = {"latitude_min": min_lat, "longitude_min": min_lon, "latitude_max": max_lat, "longitude_max": max_lon, "latitude_step": latitude_delta, "longitude_step": longitude_delta}
d["time_matrix"] = durations
d["census_tract_region_mapping"] = dict(census_tract_grids)
d["NUM_REGIONS"] = NUM_REGIONS
# with open("grid_info_multiple.json", "w") as j:
s = "grid_info_" + str(NUM_REGIONS) + ".json"
s

'grid_info_400.json'

200

In [ ]:
with open(s, "w") as j:
    json.dump(d, j)

In [ ]:
import json
with open("../austin-ems-data/grid_info_multiple.json", "r") as j:
    info = json.load(j)

In [ ]:
info["time_matrix"]